## Imports

In [121]:
import geopy.distance
from scipy.spatial import distance_matrix
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import folium
import gurobipy as gp
from gurobipy import GRB
import math
import pulp
from pyscipopt import Model, quicksum
import os
import networkx as nx
import itertools


## Downloading the us capitals distance matrix

### The distance matrix is downloaded from the following link: https://people.sc.fsu.edu/~jburkardt/datasets/cities/cities.html

#### Create distance matrix

In [11]:
# create distance matrix according to list of names of capitals
def compute_distance_matrix(capitals):
    """
    Create a distance matrix for a list of capitals. 
    The distance matrix is a 2D numpy array where the element at  position (i, j) represents the distance in km between the capitals at index i and j in the list.
    :param capitals: List of capital names
    Returns: 2D numpy array
    """
    # Geocode capitals to get latitude and longitude
    geolocator = Nominatim(user_agent="geoapiExercises") # Create a geolocator object
    coordinates = [] # List to store the coordinates of the capitals
    # Loop through the list of capitals and geocode each capital
    for capital in capitals:
        location = geolocator.geocode(capital) # Get location of the capital
        # If location is found, append the coordinates to the list
        if location:
            coordinates.append((location.latitude, location.longitude))
            print(f"Geocoded {capital}")
        # If location is not found, append None to the list
        else:
            coordinates.append((None, None))
            print(f"Could not geocode {capital}")

    # Calculate the distance matrices
    distance_mat = distance_matrix(coordinates, coordinates)

    # Convert distance matrix from kilometers to miles (1 km = 0.621371 miles)
    distance_mat_km *= 0.621371

    return distance_mat_km

#### List of capitals



In [75]:
# List of U.S. state capitals
capitals_50 = [
    "Montgomery, AL", "Juneau, AK", "Phoenix, AZ", "Little Rock, AR",
    "Sacramento, CA", "Denver, CO", "Hartford, CT", "Dover, DE",
    "Tallahassee, FL", "Atlanta, GA", "Honolulu, HI", "Boise, ID",
    "Springfield, IL", "Indianapolis, IN", "Des Moines, IA",
    "Topeka, KS", "Frankfort, KY", "Baton Rouge, LA", "Augusta, ME",
    "Annapolis, MD", "Boston, MA", "Lansing, MI", "St. Paul, MN",
    "Jackson, MS", "Jefferson City, MO", "Helena, MT", "Lincoln, NE",
    "Carson City, NV", "Concord, NH", "Trenton, NJ", "Santa Fe, NM",
    "Albany, NY", "Raleigh, NC", "Bismarck, ND", "Columbus, OH",
    "Oklahoma City, OK", "Salem, OR", "Harrisburg, PA", "Providence, RI",
    "Columbia, SC", "Pierre, SD", "Nashville, TN", "Austin, TX",
    "Salt Lake City, UT", "Montpelier, VT", "Richmond, VA",
    "Olympia, WA", "Charleston, WV", "Madison, WI", "Cheyenne, WY"
]

capitals_48 = [
    "Montgomery, AL", "Phoenix, AZ", "Little Rock, AR",
    "Sacramento, CA", "Denver, CO", "Hartford, CT", "Dover, DE",
    "Tallahassee, FL", "Atlanta, GA", "Boise, ID",
    "Springfield, IL", "Indianapolis, IN", "Des Moines, IA",
    "Topeka, KS", "Frankfort, KY", "Baton Rouge, LA", "Augusta, ME",
    "Annapolis, MD", "Boston, MA", "Lansing, MI", "St. Paul, MN",
    "Jackson, MS", "Jefferson City, MO", "Helena, MT", "Lincoln, NE",
    "Carson City, NV", "Concord, NH", "Trenton, NJ", "Santa Fe, NM",
    "Albany, NY", "Raleigh, NC", "Bismarck, ND", "Columbus, OH",
    "Oklahoma City, OK", "Salem, OR", "Harrisburg, PA", "Providence, RI",
    "Columbia, SC", "Pierre, SD", "Nashville, TN", "Austin, TX",
    "Salt Lake City, UT", "Montpelier, VT", "Richmond, VA",
    "Olympia, WA", "Charleston, WV", "Madison, WI", "Cheyenne, WY"
]


### Create distance matrices for the capitals

In [2]:


distance_matrix_50 = compute_distance_matrix(capitals_50)
distance_matrix_48 = compute_distance_matrix(capitals_48)

# Geocode capitals to get latitude and longitude
geolocator = Nominatim(user_agent="geoapiExercises") # Create a geolocator object
coordinates = [] # List to store the coordinates of the capitals
# Loop through the list of capitals and geocode each capital
for capital in capitals:
    location = geolocator.geocode(capital) # Get location of the capital
    # If location is found, append the coordinates to the list
    if location:
        coordinates.append((location.latitude, location.longitude))
        print(f"Geocoded {capital}")
    # If location is not found, append None to the list
    else:
        coordinates.append((None, None))
        print(f"Could not geocode {capital}")

# Calculate the distance matrix
distance_mat = distance_matrix(coordinates, coordinates)

# Convert distance matrix from kilometers to miles (1 km = 0.621371 miles)
distance_mat *= 0.621371

print(distance_mat.shape)


### Import distance matrices

In [122]:
distance_matrix_48 = pd.read_csv("distance_matrix_48.csv", index_col=0)
distance_matrix_48.head()
# convert it to 2d list
   # convert the 2d dataframe to a 2d list
distance_matrix_list_48 = distance_matrix_48.values.tolist()

#### Convert distance

In [4]:
def haversine(coord1, coord2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    # Convert latitude and longitude from degrees to radians
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    # Haversine formula
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c  # Output distance in kilometers
    return distance

# Calculate the distance matrix using Haversine
def calculate_distance_matrix(coordinates):
    num_cities = len(coordinates)
    dist_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            if coordinates[i] is not None and coordinates[j] is not None:
                dist_matrix[i][j] = haversine(coordinates[i], coordinates[j])
            else:
                dist_matrix[i][j] = float('inf')  # Infinite distance if no coordinates
    return dist_matrix

# Assuming 'coordinates' is a list of (latitude, longitude) tuples
distance_matrix_km = calculate_distance_matrix(coordinates)



NameError: name 'coordinates' is not defined

### Create df from the distance matrix

In [123]:


distance_df_km_48 = pd.DataFrame(distance_matrix_48, index=capitals_48, columns=capitals_48).round(2)
# print the first 4*4 of the DataFrame
print(distance_df_km_48.iloc[:4, :4])


                 Montgomery, AL  Phoenix, AZ  Little Rock, AR  Sacramento, CA
Montgomery, AL             0.00      2402.90           614.65         3240.19
Phoenix, AZ             2402.90         0.00          1824.49         1020.65
Little Rock, AR          614.65      1824.49             0.00         2628.19
Sacramento, CA          3240.19      1020.65          2628.19            0.00


#### cities coordinates

In [135]:

cities_df_48 = pd.read_csv("cities_coordinates_48.csv", index_col=0)
# print the first 4 rows of the cities_df
print(cities_df_48.head())


                         y           x
Montgomery, AL   32.366966  -86.300648
Phoenix, AZ      33.448437 -112.074141
Little Rock, AR  34.746507  -92.289627
Sacramento, CA   38.581061 -121.493895
Denver, CO       39.739236 -104.984862


## Plot of the us capitals

### Plot tour

In [144]:
# create function the as input is a tour (list of indeces) and plots the tours
def plot_tour(tour, cities_df):
    """
    Plot the cities in the tour on a map. The cities are marked with a red dot and lines are drawn between the cities in the tour.
    :param tour: list of integers representing the order of the cities in the tour include the index of the first city at the end of the list
    :param cities_df: DataFrame with the coordinates of the cities
    """
    # create a map centered at the center of the US. 
    map = folium.Map(location=[40,-95], zoom_start = 4)

    # create an empty list 'coordinates' with the same length as the number of cities in the tour
    coordinates = [None] * len(tour)
    # create list of 2-tuples 'coordinates' of the coordinates of the cities in the tour according to cities_df and add it to the list coordinates in the index city
    for city in tour:
        coordinates[city] = (cities_df.iloc[city, 0], cities_df.iloc[city, 1])
    
    # assign the coordinates of the last city as the coordinates of the first city
    coordinates[-1] = coordinates[0]

    points = []
    # create list of 2-tuples 'points' of the coordinates of the cities in the tour except the last city
    for city in tour[:-1]:
        points.append(coordinates[city])

    points.append(points[0])

    # mark the cities in the tour on the map with a red dot with the name of the city
    for city, point in enumerate(coordinates):
        if city < len(coordinates) - 1:
            name = cities_df.index[city]
            folium.Marker(point, tooltip=name, icon=folium.Icon(color='red')).add_to(map)
    # draw lines between the cities in the tour
    folium.PolyLine(points).add_to(map)
    return map

### Plot map

In [127]:
# plot a map of the us capitals with no Polyline
def plot_map(cities_df):
    """
    Plot the cities on a map. The cities are marked with a red dot.
    :param cities_df: DataFrame with the coordinates of the cities
    """
    # create a map centered at the center of the US. 
    map = folium.Map(location=[40,-95], zoom_start = 4)

    # mark the cities on the map with a red dot with the name of the city
    for city in cities_df.index:
        name = city
        folium.Marker((cities_df.loc[city, "y"], cities_df.loc[city, "x"]), tooltip=name, icon=folium.Icon(color='red')).add_to(map)
    return map

# plot the map of the 48 capitals
map_48 = plot_map(cities_df_48)
map_48

## Greedy solution

### function

In [69]:
def greedy_tsp(distance_matrix):
    """
    Implements a greedy algorithm for the traveling salesperson problem (TSP).
    
    Parameters:
        distance_matrix (DataFrame): A symmetric NxN dataframe where the entry at [i, j]
                                     represents the distance from city i to city j.
    
    Returns:
        list of tuples: Each tuple contains the start city index, the total distance of the tour,
                        and the list representing the path of the tour.
    """
    num_cities = len(distance_matrix)
    print(f'num cities: {num_cities}')
    tours = []

    # Iterate over each city to use it as the starting point of a tour
    for start_city in range(num_cities):
        visited = set()
        total_distance = 0
        current_city = start_city
        tour_path = [current_city]

        # Continue the tour until all cities are visited
        while len(visited) < num_cities:
            visited.add(current_city)
            # Find the closest unvisited city to the current city
            distances = list(distance_matrix.iloc[current_city])
            min_distance = float('inf')
            for i in range(num_cities):
                if i not in visited and distances[i] < min_distance:
                    min_distance = distances[i]
                    next_city = i

            current_city = next_city
            # add the minimum distance to the total distance if the length of visited is less than the number of cities an
            if len(visited) < num_cities: 
                total_distance += min_distance
                tour_path.append(current_city) # Add the current city to the tour path if the length of visited is less than the number of cities

        # Complete the tour by returning to the start city
        total_distance += distance_matrix.iloc[current_city, start_city]
        tour_path.append(start_city)
        tours.append((start_city, total_distance, tour_path))

    # Sort tours by the total distance
    tours.sort(key=lambda x: x[1])
    return tours





### Greedy for 48 states (without Hawaii and Alaska)

In [70]:

# Call the function
sorted_tours_km_48 = greedy_tsp(distance_df_km_48)

# Print sorted tours
for tour in sorted_tours_km_48:
    # print the start city name according to the matrix, the total distance and the path of the tour with the distance rounded to 2 decimal places
    # print(f"Start City: {distance_df_48.index[tour[0]]}, Total Distance: {tour[1]:.2f}, Path: {tour[2]}")
    # in each tour, verify that the tour length (sum of distances) is correct according to the distance matrix coparing to total distance. if it not equal print the error
    if tour[1] != sum(distance_df_km_48.iloc[tour[2][i],tour[2][i+1]] for i in range(len(tour[2])-1)):
        print("Error: Incorrect tour length")
    
    # in each tour, make sure that the start city appear twice (in the 0 place and in the last place) and all other cities appear once. if not print the error.
    if tour[2].count(tour[0]) != 2 or len(tour[2]) != len(set(tour[2])) + 1:
        print("Error: Incorrect tour path")

print(f"Shortest tour:\nStart City: {distance_df_km_48.index[sorted_tours_km_48[0][0]]}, Total Distance: {sorted_tours_km_48[0][1]:.2f}, Path: {sorted_tours_km_48[0][2]}")

num cities: 48
Shortest tour:
Start City: Phoenix, AZ, Total Distance: 20003.87, Path: [1, 28, 4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 39, 8, 0, 7, 37, 30, 21, 15, 2, 33, 40, 41, 9, 23, 44, 34, 25, 3, 1]


### plot greedy

In [225]:
plot_tour(sorted_tours_km_48[0][2], cities_df_48)

len(points): 49


## Gurobi solver

### Gurobi MTZ solution

#### Gurobi MTZ solver

In [61]:
def gurobi_mtz(distance_df):
    """
    Solves the TSP using the Miller-Tucker-Zemlin formulation.
    :param distance_df: DataFrame with the distances between cities

    Returns a tuple of:  the Start City, Total length of the tour in kilometers, and the Path as lsit of indeces (beginning and ending with the start city)
    """

    # convert the 2d dataframe to a 2d list
    c = distance_df.values.tolist()

    # Number of cities
    n = len(distance_df)
    n = len(c)
    
    # Complete graph on n vertices
    G = nx.complete_graph(n)

    # DG is directed version of G. Replaces each edge {i,j} by (i,j) and (j,i)
    DG = nx.DiGraph(G)

    # Create model object
    m = gp.Model()

    # Create variable for each edge of DG
    x = m.addVars( DG.edges, vtype=GRB.BINARY )

    # Create variable for each vertex of DG
    u = m.addVars( DG.nodes )

    # Objective function: minimize cost of tour
    m.setObjective( gp.quicksum( c[i][j] * x[i,j] for i,j in DG.edges ), GRB.MINIMIZE )

    # Leave each city once
    m.addConstrs( gp.quicksum( x[i,j] for j in DG.neighbors(i) ) == 1 for i in DG.nodes )

    # Enter each city once
    m.addConstrs( gp.quicksum( x[i,j] for i in DG.neighbors(j) ) == 1 for j in DG.nodes )

    # (original) MTZ constraints to eliminate cycles
    # m.addConstrs( u[i] - u[j] + n * x[i,j] <= n - 1 for i,j in DG.edges if j != 0 )
    m.addConstrs( u[i] - u[j] + 1 <= (n-1) * (1 - x[i,j])  for i,j in DG.edges if j != 0 )

    # Solve
    m.optimize()

    # Extract solution
    x_sol = m.getAttr('x', x)
    u_sol = m.getAttr('x', u)
    
    print(f'x_sol: {x_sol}')
    print(f'u_sol: {u_sol}')
    # Find the start city
    start_city = 0
    for i in range(n):
        if u_sol[i] > u_sol[start_city]:
            start_city = i  
    print(f'start_city: {start_city}')
    # Find the tour
    tour = [start_city]
    next_city = start_city
    while True:
        for j in DG.neighbors(next_city):
            if x_sol[next_city,j] > 0.5:
                next_city = j
                tour.append(j)
                break
        if next_city == start_city:
            break

    # Calculate the total length of the tour
    total_length = sum(c[tour[i]][tour[i+1]] for i in range(len(tour)-1))
    total_length += c[tour[-1]][tour[0]]

    print("Objective:",m.objVal)

    
    return start_city, total_length, tour

#### Gurobi MTZ for 48 states (without Hawaii and Alaska)

In [62]:
mtz_start_city, mtz_total_length, mtz_tour = gurobi_mtz(distance_df_km_48)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[rosetta2] - Darwin 21.6.0 21G646)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2305 rows, 2304 columns and 11139 nonzeros
Model fingerprint: 0x3b05cf52
Variable types: 48 continuous, 2256 integer (2256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 0 rows and 1 columns
Presolve time: 0.02s
Presolved: 2305 rows, 2303 columns, 11092 nonzeros
Variable types: 47 continuous, 2256 integer (2256 binary)

Root relaxation: objective 1.367100e+04, 155 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 13670.9953    0   81          - 13670.9953      -     -    0s
     0     0 16

##### Plot MTZ tour

In [65]:
# plot tour of gurobi_mtz
plot_tour(mtz_tour, cities_df_48)

len(points): 49


### Gurobi DFJ soultion

#### GUROBI DJF solver with lazy constraints

In [ ]:
def gurobi_dfj_lazy(distance_df):
    """
    Solves the TSP using the Dantzig-Fulkerson-Johnson formulation.
    :param distance_df: DataFrame with the distances between cities

    Returns a tuple of:  the Start City, Total length of the tour in kilometers, and the Path as lsit of indeces (beginning and ending with the start city)
    """

    # convert the 2d dataframe to a 2d list
    c = distance_df.values.tolist()

    # Number of cities
    n = len(distance_df)
    n = len(c)
    r = 0 # Start city as arbitrary index

    # Complete graph on n vertices
    G = nx.complete_graph(n)

    # DG is directed version of G. Replaces each edge {i,j} by (i,j) and (j,i)
    DG = nx.DiGraph(G)

    # Create model object
    m = gp.Model()

    # Create variable for each edge of DG
    x = m.addVars( DG.edges, vtype=GRB.BINARY )

    # Objective function: minimize cost of tour
    m.setObjective( gp.quicksum( c[i][j] * x[i,j] for i,j in DG.edges ), GRB.MINIMIZE )

    # Leave each city once
    m.addConstrs( gp.quicksum( x[i,j] for j in DG.neighbors(i) ) == 1 for i in DG.nodes )

    # Enter each city once
    m.addConstrs( gp.quicksum( x[i,j] for i in DG.neighbors(j) ) == 1 for j in DG.nodes )
 

    m._DG = DG
    m._x = x
    m._r = r

    m.update()

    # Add (violated) DFJ constraints in a callback routine
    def DFJ_callback(m, where):
        
        # check if LP relaxation at this BB node is integer
        if where == GRB.Callback.MIPSOL: 
            
            # retrieve the LP relaxation solution at this BB node
            xval = m.cbGetSolution(m._x)
            
            # which edges are selected in the LP solution?
            chosen_edges = [ (i,j) for i,j in m._DG.edges if xval[i,j] > 0.5 ]
            
            # if the solution is not a tour, it will have multiple subtours
            for component in nx.weakly_connected_components( DG.edge_subgraph(chosen_edges) ):
                
                if len(component) < n:
                    
                    # must pick fewer than |component| interior edges
                    interior_edges = nx.edge_boundary( m._DG, component, component )
                    
                    m.cbLazy( gp.quicksum( m._x[i,j] for i,j in interior_edges ) <= len(component) - 1 )
                    
    # Tell Gurobi that we will be adding (lazy) constraints
    m.Params.lazyConstraints = 1

    # Designate the callback routine to be the function "cut_callback"
    m._callback = DFJ_callback

    # Solve the MIP with our callback
    m.optimize(m._callback)

    # Extract solution
    x_sol = m.getAttr('x', x)
    
    print(f'x_sol: {x_sol}')
    # Find the start city
    start_city = 0

    print(f'start_city: {start_city}')
    # Find the tour
    tour = [start_city]
    next_city = start_city
    while True:
        for j in DG.neighbors(next_city):
            if x_sol[next_city,j] > 0.5:
                next_city = j
                tour.append(j)
                break
        if next_city == start_city:
            break

    # Calculate the total length of the tour
    total_length = sum(c[tour[i]][tour[i+1]] for i in range(len(tour)-1))
    total_length += c[tour[-1]][tour[0]]

    print("Objective:",m.objVal)

    tour_edges = [ (i,j) for i,j in DG.edges if x[i,j].x > 0.5 ]

    # Get a tour by picking a chosen edge (t,s) and 
    #   finding a shortest s,t-path in G[chosen_edges\(t,s)]

    (t,s) = tour_edges.pop()

    paths = nx.single_source_shortest_path( G.edge_subgraph(tour_edges), s )

    print("Tour:",paths[t])

    
    return start_city, total_length, tour



#### GUROBI DFJ solver for 48 states (without Hawaii and Alaska)

In [ ]:
dfj_start_city, dfj_total_length, dfj_tour = gurobi_dfj_lazy(distance_df_km_48)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[rosetta2] - Darwin 21.6.0 21G646)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 96 rows, 2256 columns and 4512 nonzeros
Model fingerprint: 0xea4966a8
Variable types: 0 continuous, 2256 integer (2256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 96 rows, 2256 columns, 4512 nonzeros
Variable types: 0 continuous, 2256 integer (2256 binary)

Root relaxation: objective 1.584608e+04, 111 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 15846.0800    0   24          - 15846.0800      -     -    0s
     0     0 16460.1

#### Plot DFJ tour

In [ ]:
plot_tour(dfj_tour, cities_df_48)

len(points): 49


## Direct CBC

### assign cbc solver

In [101]:
# assign solver to use with parameters max_iterations and time_limit
def assign_cbc_solver(time_limit=None, presolve=None, cuts=None, strong=None):
    """
    Assign the solver to use for optimization.
    :param time_limit: Time limit for the optimization
    :param presolve: Whether to use presolve (0 = no, 1 = yes). presolve is used to simplify the problem before solving it by removing redundant constraints and variables.
    :param cuts: Whether to use cuts (0 = no, 1 = yes). Cuts are linear inequalities that are added to the problem to improve the lower bound.
    :param strong: Whether to use strong branching (0 = no, 1 = yes). Strong branching is a method used to select the best branching variable at each node in the branch-and-bound algorithm.
    """
    solver = pulp.PULP_CBC_CMD(mip=1, msg=1, timeLimit=time_limit)
    # solver = pulp.PULP_CBC_CMD(mip=1, msg=1, timeLimit=time_limit, cuts=cuts, strong=strong)
    # solver.options = [max_iterations, time_limit]
    return solver

### MTZ

#### algorithm

In [143]:
def solve_tsp_mtz(distances, solver):
    """
    Solve the Traveling Salesman Problem (TSP) using the Miller-Tucker-Zemlin (MTZ) formulation.
    :param distances: 2D list of distances between cities
    :param solver: Solver to use for optimization. must be according to 'pulp' library
    """
    n = len(distances)  # Number of cities
    cities = range(n)
    
    # Create the problem variable to contain the problem data
    model = pulp.LpProblem("TSP", pulp.LpMinimize)
    
    # Decision variables
    x = pulp.LpVariable.dicts("x", (cities, cities), 0, 1, pulp.LpBinary)
    u = pulp.LpVariable.dicts("u", cities, 0, n-1, pulp.LpInteger)
    
    # Objective function: Minimize the total distance traveled
    model += pulp.lpSum(distances[i][j] * x[i][j] for i in cities for j in cities if i != j)
    
    # Constraints
    # Each city has exactly one departure
    for i in cities:
        model += pulp.lpSum(x[i][j] for j in cities if i != j) == 1
    
    # Each city has exactly one arrival
    for j in cities:
        model += pulp.lpSum(x[i][j] for i in cities if i != j) == 1
    
    # Subtour elimination (MTZ constraints)
    for i in cities:
        for j in cities:
            if i != j and i != 0 and j != 0:
                model += u[i] - u[j] + n * x[i][j] <= n - 1
    

    # Solve the problem
    model.solve(solver)
    
    # Extract the tour
    tour = []
    for i in cities:
        for j in cities:
            if pulp.value(x[i][j]) == 1:
                tour.append((i, j))

    for constraint in model.constraints:
        print(f'constraints:\n',model.constraints[constraint].name, model.constraints[constraint].slack)
    
    #  organize the tour such that the second element of the first tuple is the first element of the second tuple and so on. the first elemt of the first tuple is the start city and the last element of the last tuple is the start city
    # do it by iterating over every element in the tour . Do nother inner iteration to find the next step - the tuple that first element of it is equal to the second element of the current tuple - should follow the element of the current tuple. if the second element of the current tuple is equal to the  of the next tuple, appent the the next tuple of the current tuple and continue to the next iteration
    new_tour = [tour.pop(0)]
    # while the original tour is not empty
    while tour:
        for i in range(len(tour)):
            if new_tour[-1][1] == tour[i][0]:
                new_tour.append(tour.pop(i))
                break
    
    # convert the tour to a list of indeces
    new_tour = [step[0] for step in new_tour]

    # Add the start city to the end of the tour
    new_tour.append(new_tour[0])

    return new_tour, pulp.value(model.objective)






##### Example usage with 4*4 matrix

In [64]:
# Example usage (assuming distances is a 2D list of distances between cities)
distances = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

# Create the solver
solver = assign_cbc_solver(time_limit=60)

tour, obj_val = solve_tsp_mtz(distances, solver)
print("Tour:", tour)
print("Total Distance:", obj_val)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/orrzwebner/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fm/n09gxskd7bx0_sj6vdt2m1fw0000gn/T/7930d51773cc40198ff323e5954ccce8-pulp.mps -sec 60 -timeMode elapsed -branch -printingOptions all -solution /var/folders/fm/n09gxskd7bx0_sj6vdt2m1fw0000gn/T/7930d51773cc40198ff323e5954ccce8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 19 COLUMNS
At line 104 RHS
At line 119 BOUNDS
At line 135 ENDATA
Problem MODEL has 14 rows, 15 columns and 42 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
Option for timeMode changed from cpu to elapsed
Continuous objective value is 80 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0004I processed model has 14 rows, 15 columns (15 integer (12 of which binary)) and 48 elements
Cutoff increment increase

#### solve CBC MTZ tour for 48 states

In [114]:
solver = assign_cbc_solver(time_limit=3600, cuts=1, strong=1)
distances = distance_matrix_list_48
mtz_cbc_tour, mtz_cbc_obj = solve_tsp_mtz(distances, solver)
print("Tour:", mtz_cbc_tour)
print("Total Distance:", mtz_cbc_obj)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/orrzwebner/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fm/n09gxskd7bx0_sj6vdt2m1fw0000gn/T/c6526fd21fd64047b4acecf6182ee65f-pulp.mps -sec 3600 -timeMode elapsed -branch -printingOptions all -solution /var/folders/fm/n09gxskd7bx0_sj6vdt2m1fw0000gn/T/c6526fd21fd64047b4acecf6182ee65f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2263 COLUMNS
At line 20124 RHS
At line 22383 BOUNDS
At line 24687 ENDATA
Problem MODEL has 2258 rows, 2303 columns and 10998 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 3600
Option for timeMode changed from cpu to elapsed
Continuous objective value is 13668.9 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2162 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2162 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0

#### plot CBC MTZ tour

In [145]:
plot_tour(mtz_cbc_tour, cities_df_48)

### DFJ

#### Algorithm for DFJ

In [146]:
def solve_tsp_dfj(distances, solver):
    """
    Solve the Traveling Salesman Problem (TSP) using the Miller-Tucker-Zemlin (MTZ) formulation.
    :param distances: 2D list of distances between cities
    :param solver: Solver to use for optimization. must be according to 'pulp' library
    """
    n = len(distances)  # Number of cities
    cities = range(n)
    
    # Create the problem variable to contain the problem data
    model = pulp.LpProblem("TSP", pulp.LpMinimize)
    
    # Decision variables
    x = pulp.LpVariable.dicts("x", (cities, cities), 0, 1, pulp.LpBinary)
    u = pulp.LpVariable.dicts("u", cities, 0, n-1, pulp.LpInteger)
    
    # Objective function: Minimize the total distance traveled
    model += pulp.lpSum(distances[i][j] * x[i][j] for i in cities for j in cities if i != j)
    
    # Constraints
    # Each city has exactly one departure
    for i in cities:
        model += pulp.lpSum(x[i][j] for j in cities if i != j) == 1
    
    # Each city has exactly one arrival
    for j in cities:
        model += pulp.lpSum(x[i][j] for i in cities if i != j) == 1
    
    
    # Subtour elimination (DFJ constraints)
    for S in range(2, n):  # For each subset of cities from size 2 to n-1
        for subset in pulp.allcombinations(cities, S):
            model += pulp.lpSum(x[i][j] for i in subset for j in subset if i != j) <= len(subset) - 1 # Eliminate subtours of size S by adding a constraint that the number of edges in the subset is less than the number of vertices in the subset
    

    # Solve the problem
    model.solve(solver)
    
    # Extract the tour
    tour = []
    for i in cities:
        for j in cities:
            if pulp.value(x[i][j]) == 1:
                tour.append((i, j))

    for constraint in model.constraints:
        print(f'constraints:\n',model.constraints[constraint].name, model.constraints[constraint].slack)
    
    # organize the tour such that the second element of the first tuple is the first element of the second tuple and so on. the first elemt of the first tuple is the start city and the last element of the last tuple is the start city
    new_tour = [tour.pop(0)]
    # while the original tour is not empty
    while tour:
        for i in range(len(tour)):
            if new_tour[-1][1] == tour[i][0]:
                new_tour.append(tour.pop(i))
                break
    
    # convert the tour to a list of indeces
    new_tour = [step[0] for step in new_tour]

    # Add the start city to the end of the tour
    new_tour.append(new_tour[0])

    return new_tour, pulp.value(model.objective)
    

#### Solve CBC DFJ tour for 48 states

In [116]:
solver = assign_cbc_solver(time_limit=3600, cuts=1, strong=1)
dfj_cbc_tour, dfj_cbc_obj = solve_tsp_mtz(distance_matrix_list_48, solver)
print("Tour:", dfj_cbc_tour)
print("Total Distance:", dfj_cbc_obj)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/orrzwebner/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fm/n09gxskd7bx0_sj6vdt2m1fw0000gn/T/cbf4e832906e4f37aedb7d370f8f7616-pulp.mps -sec 3600 -timeMode elapsed -branch -printingOptions all -solution /var/folders/fm/n09gxskd7bx0_sj6vdt2m1fw0000gn/T/cbf4e832906e4f37aedb7d370f8f7616-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2263 COLUMNS
At line 20124 RHS
At line 22383 BOUNDS
At line 24687 ENDATA
Problem MODEL has 2258 rows, 2303 columns and 10998 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 3600
Option for timeMode changed from cpu to elapsed
Continuous objective value is 13668.9 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2162 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2162 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0

#### Plot CBC DFJ tour

In [151]:
plot_tour(dfj_cbc_tour, cities_df_48)

### GUROBI DJF solver without lazy conditions

In [ ]:
def gurobi_dfj(distance_df):
    """
    Solves the TSP using the Dantzig-Fulkerson-Johnson formulation.
    :param distance_df: DataFrame with the distances between cities

    Returns a tuple of:  the Start City, Total length of the tour in kilometers, and the Path as lsit of indeces (beginning and ending with the start city)
    """

    # convert the 2d dataframe to a 2d list
    c = distance_df.values.tolist()

    # Number of cities
    n = len(c)
    cities = range(n)
    r = 0 # Start city as arbitrary index

    # Complete graph on n vertices
    G = nx.complete_graph(n)

    # DG is directed version of G. Replaces each edge {i,j} by (i,j) and (j,i)
    DG = nx.DiGraph(G)

    # Create model object
    m = gp.Model()

    # Create variable for each edge of DG
    x = m.addVars( DG.edges, vtype=GRB.BINARY )

    # Objective function: minimize cost of tour
    m.setObjective( gp.quicksum( c[i][j] * x[i,j] for i,j in DG.edges ), GRB.MINIMIZE )

    # Leave each city once
    m.addConstrs( gp.quicksum( x[i,j] for j in DG.neighbors(i) ) == 1 for i in DG.nodes )

    # Enter each city once
    m.addConstrs( gp.quicksum( x[i,j] for i in DG.neighbors(j) ) == 1 for j in DG.nodes )

    # Subtour elimination (DFJ constraints)
    for S in range(2, n):  # For each subset of cities from size 2 to n-1
        # for subset in pulp.allcombinations(cities, S):
        for subset in itertools.combinations(cities, S):
            # model += pulp.lpSum(x[i][j] for i in subset for j in subset if i != j) <= len(subset) - 1 # Eliminate subtours of size S by adding a constraint that the number of edges in the subset is less than the number of vertices in the subset
            m.addConstr( gp.quicksum( x[i,j] for i in subset for j in subset if i != j ) <= len(subset) - 1 )

    # Solve
    m.optimize()

    # Extract solution
    x_sol = m.getAttr('x', x)
    
    print(f'x_sol: {x_sol}')
    # Find the start city
    start_city = 0

    print(f'start_city: {start_city}')
    # Find the tour
    tour = [start_city]
    next_city = start_city
    while True:
        for j in DG.neighbors(next_city):
            if x_sol[next_city,j] > 0.5:
                next_city = j
                tour.append(j)
                break
        if next_city == start_city:
            break

    # Calculate the total length of the tour
    total_length = sum(c[tour[i]][tour[i+1]] for i in range(len(tour)-1))
    total_length += c[tour[-1]][tour[0]]

    print("Objective:",m.objVal)

    tour_edges = [ (i,j) for i,j in DG.edges if x[i,j].x > 0.5 ]

    # Get a tour by picking a chosen edge (t,s) and 
    #   finding a shortest s,t-path in G[chosen_edges\(t,s)]

    (t,s) = tour_edges.pop()

    paths = nx.single_source_shortest_path( G.edge_subgraph(tour_edges), s )

    print("Tour:",paths[t])

    
    return start_city, total_length, tour

